In [19]:
#import statemnts
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


In [2]:
#reading the matches.csv file into a pd dataFrame
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2022,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2022,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2022,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2022,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2022,Manchester City


In [3]:
#investigating missing data
matches.shape

(1520, 27)

In [4]:
38 * 20 * 2
matches.team.value_counts()
#so we conclude their is no missing data

team
Manchester City             76
Crystal Palace              76
Nottingham Forest           76
Brentford                   76
Everton                     76
Wolverhampton Wanderers     76
Fulham                      76
Arsenal                     76
Brighton and Hove Albion    76
Bournemouth                 76
West Ham United             76
Manchester United           76
Newcastle United            76
Chelsea                     76
Tottenham Hotspur           76
Aston Villa                 76
Liverpool                   76
Luton Town                  38
Burnley                     38
Sheffield United            38
Leicester City              38
Leeds United                38
Southampton                 38
Name: count, dtype: int64

In [5]:
matches["round"].value_counts()

round
Matchweek 1     40
Matchweek 30    40
Matchweek 23    40
Matchweek 24    40
Matchweek 25    40
Matchweek 18    40
Matchweek 26    40
Matchweek 27    40
Matchweek 28    40
Matchweek 31    40
Matchweek 2     40
Matchweek 32    40
Matchweek 33    40
Matchweek 29    40
Matchweek 35    40
Matchweek 36    40
Matchweek 37    40
Matchweek 34    40
Matchweek 22    40
Matchweek 21    40
Matchweek 20    40
Matchweek 19    40
Matchweek 3     40
Matchweek 4     40
Matchweek 5     40
Matchweek 6     40
Matchweek 7     40
Matchweek 8     40
Matchweek 9     40
Matchweek 10    40
Matchweek 11    40
Matchweek 12    40
Matchweek 13    40
Matchweek 14    40
Matchweek 15    40
Matchweek 16    40
Matchweek 17    40
Matchweek 38    40
Name: count, dtype: int64

In [6]:
#cleaning our data for model building
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [7]:
matches.date = pd.to_datetime(matches.date)

In [8]:
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                       int64
ga                       int64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
season                   int64
team                    object
dtype: object

In [9]:
#creating predictors for machine learning
del matches["comp"]
del matches["notes"]

In [11]:
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")
matches

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-11,20:00,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,...,0.0,0,0,2022,Manchester City,0,5,20,4,1
3,2023-08-19,20:00,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,...,0.0,0,0,2022,Manchester City,1,16,20,5,1
4,2023-08-27,14:00,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,...,2.0,0,1,2022,Manchester City,0,18,14,6,1
5,2023-09-02,15:00,Matchweek 4,Sat,Home,W,5,1,Fulham,2.2,...,0.0,1,1,2022,Manchester City,1,9,15,5,1
6,2023-09-16,15:00,Matchweek 5,Sat,Away,W,3,1,West Ham,3.6,...,1.0,0,0,2022,Manchester City,0,21,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,0.8,...,0.0,0,0,2021,Southampton,0,16,14,6,0
43,2023-05-08,20:00,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,3.1,...,0.0,1,1,2021,Southampton,0,17,20,0,0
44,2023-05-13,15:00,Matchweek 36,Sat,Home,L,0,2,Fulham,0.4,...,0.0,0,0,2021,Southampton,1,9,15,5,0
45,2023-05-21,14:00,Matchweek 37,Sun,Away,L,1,3,Brighton,0.6,...,1.0,0,0,2021,Southampton,0,4,14,6,0


In [13]:
#creating our initial machine learning model
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']

In [14]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [15]:
rf.fit(train[predictors], train["target"])
preds = rf.predict(test[predictors])

In [17]:
error = accuracy_score(test["target"], preds)
error

0.6229508196721312

In [20]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,168,61
1,77,60


In [21]:
precision_score(test["target"], preds)

0.49586776859504134

In [25]:
#Improve precision with rolling averages
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")
group

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2022-08-07,16:30,Matchweek 1,Sun,Away,W,2,0,West Ham,2.2,...,1.0,1,1,2021,Manchester City,0,21,16,6,1
2,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4,0,Bournemouth,1.7,...,0.0,0,0,2021,Manchester City,1,2,15,5,1
3,2022-08-21,16:30,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,2.1,...,1.0,0,0,2021,Manchester City,0,16,16,6,0
4,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,2.2,...,0.0,0,0,2021,Manchester City,1,7,15,5,1
5,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,3.3,...,0.0,0,0,2021,Manchester City,1,17,19,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-04-28,16:30,Matchweek 35,Sun,Away,W,2,0,Nott'ham Forest,0.9,...,1.0,0,0,2022,Manchester City,0,17,16,6,1
52,2024-05-04,17:30,Matchweek 36,Sat,Home,W,5,1,Wolves,3.7,...,0.0,2,2,2022,Manchester City,1,22,17,5,1
53,2024-05-11,12:30,Matchweek 37,Sat,Away,W,4,0,Fulham,3.2,...,0.0,1,1,2022,Manchester City,0,9,12,5,1
54,2024-05-14,20:00,Matchweek 34,Tue,Away,W,2,0,Tottenham,2.4,...,0.0,1,1,2022,Manchester City,0,20,20,1,1


In [24]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [26]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
4,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,2.2,...,5,1,3.000000,1.000000,17.666667,6.000000,17.466667,0.666667,0.333333,0.333333
5,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,3.3,...,2,1,3.666667,1.666667,19.333333,7.333333,15.933333,0.333333,0.000000,0.000000
6,2022-09-03,17:30,Matchweek 6,Sat,Away,D,1,1,Aston Villa,2.1,...,5,0,4.333333,1.666667,18.666667,8.000000,15.033333,0.333333,0.000000,0.000000
9,2022-09-17,12:30,Matchweek 8,Sat,Away,W,3,0,Wolves,1.1,...,5,1,3.666667,1.000000,16.000000,6.000000,15.233333,0.333333,0.000000,0.000000
10,2022-10-02,14:00,Matchweek 9,Sun,Home,W,6,3,Manchester Utd,3.2,...,6,1,3.333333,0.333333,15.333333,6.666667,17.000000,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-04-28,16:30,Matchweek 35,Sun,Away,W,2,0,Nott'ham Forest,0.9,...,6,1,4.333333,1.000000,22.666667,8.666667,16.666667,0.333333,0.333333,0.333333
52,2024-05-04,17:30,Matchweek 36,Sat,Home,W,5,1,Wolves,3.7,...,5,1,3.666667,0.333333,20.333333,7.666667,17.533333,0.666667,0.333333,0.333333
53,2024-05-11,12:30,Matchweek 37,Sat,Away,W,4,0,Fulham,3.2,...,5,1,3.666667,0.333333,14.333333,7.000000,17.000000,0.666667,0.666667,0.666667
54,2024-05-14,20:00,Matchweek 34,Tue,Away,W,2,0,Tottenham,2.4,...,1,1,3.666667,0.333333,14.666667,7.666667,17.200000,0.333333,1.000000,1.000000


In [28]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling

/var/folders/9m/63w8xdt15_nf2852_3g2q8wc0000gn/T/ipykernel_41648/1738758534.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


date   time         round  day venue result  \
team                                                                           
Arsenal                 3  2022-08-27  17:30   Matchweek 4  Sat  Home      W   
                        4  2022-08-31  19:30   Matchweek 5  Wed  Home      W   
                        5  2022-09-04  16:30   Matchweek 6  Sun  Away      L   
                        7  2022-09-18  12:00   Matchweek 8  Sun  Away      W   
                        8  2022-10-01  12:30   Matchweek 9  Sat  Home      W   
...                               ...    ...           ...  ...   ...    ...   
Wolverhampton Wanderers 40 2024-04-24  19:45  Matchweek 29  Wed  Home      L   
                        41 2024-04-27  15:00  Matchweek 35  Sat  Home      W   
                        42 2024-05-04  17:30  Matchweek 36  Sat  Away      L   
                        43 2024-05-11  15:00  Matchweek 37  Sat  Home      L   
                        44 2024-05-19  16:00  Matchweek 38  Sun  Away      L   

                            gf  ga         opponent   xg  ...  day_code  \
team                                                      ...             
Arsenal                 3    2   1           Fulham  2.6  ...         5   
                        4    2   1      Aston Villa  2.4  ...         2   
                        5    1   3   Manchester Utd  1.3  ...         6   
                        7    3   0        Brentford  1.5  ...         6   
                        8    3   1        Tottenham  2.4  ...         5   
...                         ..  ..              ...  ...  ...       ...   
Wolverhampton Wanderers 40   0   1      Bournemouth  0.5  ...         2   
                        41   2   1       Luton Town  1.2  ...         5   
                        42   1   5  Manchester City  0.3  ...         5   
                        43   1   3   Crystal Palace  1.6  ...         5   
                        44   0   2        Liverpool  0.5  ...         6   

                            target  gf_rolling ga_rolling sh_rolling  \
team                                                                   
Arsenal                 3        1    3.000000   0.666667  14.333333   
                        4        1    3.000000   1.000000  18.333333   
                        5        0    2.333333   0.666667  19.333333   
                        7        1    1.666667   1.666667  20.000000   
                        8        1    2.000000   1.333333  17.000000   
...                            ...         ...        ...        ...   
Wolverhampton Wanderers 40       0    1.000000   2.000000   9.666667   
                        41       1    0.666667   1.666667  10.333333   
                        42       0    0.666667   1.333333  11.000000   
                        43       0    1.000000   2.333333  10.000000   
                        44       0    1.333333   3.000000   9.666667   

                           sot_rolling dist_rolling  fk_rolling  pk_rolling  \
team                                                                          
Arsenal                 3     5.000000    14.133333    0.333333    0.000000   
                        4     7.000000    14.433333    0.333333    0.000000   
                        5     7.333333    15.533333    0.666667    0.000000   
                        7     6.333333    16.800000    1.000000    0.000000   
                        8     6.000000    17.700000    0.666667    0.000000   
...                                ...          ...         ...         ...   
Wolverhampton Wanderers 40    4.000000    20.533333    0.333333    0.333333   
                        41    3.333333    18.766667    0.000000    0.000000   
                        42    4.000000    19.666667    0.000000    0.000000   
                        43    3.333333    15.966667    0.000000    0.000000   
                        44    4.666667    14.133333    0.000000    0.000000   

                            pkatt_rolling  
team   

In [29]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2,1,Fulham,2.6,...,5,1,3.000000,0.666667,14.333333,5.000000,14.133333,0.333333,0.000000,0.000000
1,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2,1,Aston Villa,2.4,...,2,1,3.000000,1.000000,18.333333,7.000000,14.433333,0.333333,0.000000,0.000000
2,2022-09-04,16:30,Matchweek 6,Sun,Away,L,1,3,Manchester Utd,1.3,...,6,0,2.333333,0.666667,19.333333,7.333333,15.533333,0.666667,0.000000,0.000000
3,2022-09-18,12:00,Matchweek 8,Sun,Away,W,3,0,Brentford,1.5,...,6,1,1.666667,1.666667,20.000000,6.333333,16.800000,1.000000,0.000000,0.000000
4,2022-10-01,12:30,Matchweek 9,Sat,Home,W,3,1,Tottenham,2.4,...,5,1,2.000000,1.333333,17.000000,6.000000,17.700000,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,2024-04-24,19:45,Matchweek 29,Wed,Home,L,0,1,Bournemouth,0.5,...,2,0,1.000000,2.000000,9.666667,4.000000,20.533333,0.333333,0.333333,0.333333
1447,2024-04-27,15:00,Matchweek 35,Sat,Home,W,2,1,Luton Town,1.2,...,5,1,0.666667,1.666667,10.333333,3.333333,18.766667,0.000000,0.000000,0.000000
1448,2024-05-04,17:30,Matchweek 36,Sat,Away,L,1,5,Manchester City,0.3,...,5,0,0.666667,1.333333,11.000000,4.000000,19.666667,0.000000,0.000000,0.000000
1449,2024-05-11,15:00,Matchweek 37,Sat,Home,L,1,3,Crystal Palace,1.6,...,5,0,1.000000,2.333333,10.000000,3.333333,15.966667,0.000000,0.000000,0.000000


In [31]:
#Retraining our machine learning model
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [32]:
error

0.5537190082644629

In [33]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)


In [34]:
combined.head(10)


,actual,predicted,date,team,opponent,result
55,1,1,2024-01-20,Arsenal,Crystal Palace,W
56,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
57,1,1,2024-02-04,Arsenal,Liverpool,W
58,1,0,2024-02-11,Arsenal,West Ham,W
59,1,0,2024-02-17,Arsenal,Burnley,W
60,1,1,2024-02-24,Arsenal,Newcastle Utd,W
61,1,1,2024-03-04,Arsenal,Sheffield Utd,W
62,1,1,2024-03-09,Arsenal,Brentford,W
63,0,0,2024-03-31,Arsenal,Manchester City,D
64,1,1,2024-04-03,Arsenal,Luton Town,W


In [35]:
#combining home and away predictions
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [36]:
combined["new_team"] = combined["team"].map(mapping)
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
2,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal,0,0,Liverpool,Arsenal,L,Liverpool
3,1,0,2024-02-11,Arsenal,West Ham,W,Arsenal,0,0,West Ham United,Arsenal,L,West Ham
4,1,0,2024-02-17,Arsenal,Burnley,W,Arsenal,0,0,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0,0,2024-04-24,Wolverhampton Wanderers,Bournemouth,L,Wolves,1,0,Bournemouth,Wolves,W,Bournemouth
326,1,0,2024-04-27,Wolverhampton Wanderers,Luton Town,W,Wolves,0,0,Luton Town,Wolves,L,Luton Town
327,0,0,2024-05-04,Wolverhampton Wanderers,Manchester City,L,Wolves,1,1,Manchester City,Wolves,W,Manchester City
328,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L,Wolves,1,1,Crystal Palace,Wolves,W,Crystal Palace


In [37]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()


actual_x
1    59
0    38
Name: count, dtype: int64

In [2]:
59/38+59

60.55263157894737